Goal: Build language model.

References:
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
  - https://github.com/karpathy/char-rnn
  - https://cs.stanford.edu/people/karpathy/char-rnn/
  - https://gist.github.com/karpathy/587454dc0146a6ae21fc
- https://www.tensorflow.org/text/tutorials/text_generation

In [ ]:
import tensorflow as tf
import numpy as np
ks = tf.keras
print("TensorFlow version:", tf.__version__)

import urllib
import math
from collections import namedtuple

In [ ]:
# I define `dot_product_attention` in this notebook below. No need to import it.

# Install the tensor2tensor library which contains useful functions for the attention mechanism.
## !pip3 install tensor2tensor
## from tensor2tensor.layers.common_attention import dot_product_attention

# Get and process the data

Using a Shakespeare dataset

In [ ]:
# Karpathy's datasets used in his blog post,
# http://karpathy.github.io/2015/05/21/rnn-effectiveness/,
# and listed here: https://cs.stanford.edu/people/karpathy/char-rnn/.

TEXT_URL = {
    'shakespeare': 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt',
    'linux': 'https://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt',
    'tolstoy': 'https://cs.stanford.edu/people/karpathy/char-rnn/warpeace_input.txt',
}['shakespeare']  # Select a dataset

with urllib.request.urlopen(TEXT_URL) as f:
  text = f.read()

print(f'Length of text: {len(text)} characters')

In [ ]:
# Note that the text is stored as a byte string
print(type(text))

In [ ]:
# look at sample of the data
print(text[2100:2600].decode("utf-8"))

In [ ]:
# This will be character level model. Serious language models use word pieces (https://paperswithcode.com/method/wordpiece).

# Make a numpy array of ASCII chars
raw_seq = np.frombuffer(text, dtype=np.uint8)

# Token ID to ascii code conversion
token_to_ascii = np.array(sorted(set(raw_seq)))
VOCAB_SIZE = len(token_to_ascii)

# Ascii code to token ID conversion
ascii_to_token = np.full(256, -1, np.int_)
for token, ascii in enumerate(token_to_ascii):
  ascii_to_token[ascii] = token

# Convert ascii array to token ID array
token_seq = ascii_to_token[raw_seq]

print('vocab size:',VOCAB_SIZE)
print('seq:', token_seq[:20])
print(token_seq.shape)
print(token_seq.dtype)
print('\ntoken_to_char:', token_to_ascii)
print('any invalid?', np.any(token_seq == -1))
print('min:', np.min(token_seq),'  max:', np.max(token_seq))

In [ ]:
BATCH_SIZE = 32  # 16
CONTEXT_SIZE = 100  #  500;  truncated sequence length
PAD_CHAR = token_to_ascii[0]
PAD_LEN = math.ceil(token_seq.size / (BATCH_SIZE*CONTEXT_SIZE)) * BATCH_SIZE*CONTEXT_SIZE - token_seq.size

parallel_seq = np.append(token_seq, [PAD_CHAR]*PAD_LEN).reshape(BATCH_SIZE, -1)

# pad with beginning of sequences from next row
full_batches = 2  # How many full batches end of each row should bleed into start of next row
parallel_seq = np.concatenate((parallel_seq, np.roll(parallel_seq[:,:CONTEXT_SIZE*full_batches+1],-1,0)),1)
print('shape:', parallel_seq.shape)

NUM_BATCHES = (parallel_seq.shape[1]-1) // CONTEXT_SIZE
print('num batches:', NUM_BATCHES)
print('assert',parallel_seq.size - NUM_BATCHES*BATCH_SIZE*CONTEXT_SIZE,'==',BATCH_SIZE)

In [ ]:
def get_batch(batch_i, offset=0):
  # When offset==0 we have a training batch, and when offset==1 we have the training targets
  return parallel_seq[:, batch_i*CONTEXT_SIZE+offset: (batch_i+1)*CONTEXT_SIZE+offset]

# get an example batch
print(get_batch(0))
print('')
print(get_batch(NUM_BATCHES-1))

In [ ]:
# Human readable render of first training batch
[row.tobytes().decode('utf8') for row in token_to_ascii[get_batch(0)]]

In [ ]:
# Show training targets for the above batch
[row.tobytes().decode('utf8') for row in token_to_ascii[get_batch(0, offset=1)]]

In [ ]:
# Second to last training batch. Each line is now the next line down in the first batch
[row.tobytes().decode('utf8') for row in token_to_ascii[get_batch(NUM_BATCHES-2)]]

# Define the model

In [ ]:
# Define our model

# TODO: implement cells myself
CELL_CLS = {
    'rnn': ks.layers.SimpleRNNCell,
    'lstm': ks.layers.LSTMCell,
    'gru': ks.layers.GRUCell,
}['lstm']

class RNN(ks.Model):

  def __init__(self, use_cnn=False):
    super(RNN, self).__init__()
    self.cells = [CELL_CLS(100)]  # , CELL_CLS(50)]
    self.embedding_size = 20
    self.input_embed = ks.layers.Dense(self.embedding_size)
    self.output_stack = [ks.layers.Dense(VOCAB_SIZE)]
    self.conv1d = ks.layers.Conv1D(filters=self.embedding_size, kernel_size=4, padding='causal')  # 'causal' convolutions only depend on inputs to the left (and center) of the current position
    self.use_cnn = use_cnn

  def call(self, x, s=None, cache=None):
    # `x` is the input tensor.
    # `s` is thee initial recurrent state. If None then a zero tensor is used.

    # Expecting x.shape == (batch_size, context_size), where batch_size and context_size can be variable from run to run
    bs, cs = tf.unstack(tf.shape(x))
    x = tf.one_hot(x, VOCAB_SIZE)  # shape == (batch_size, context_size, VOCAB_SIZE), where VOCAB_SIZE is a global constant

    if s is None:
      s = [cell.get_initial_state(batch_size=bs, dtype=tf.float32) for cell in self.cells]
    else:
      s = list(s)  # Make a copy of the input list since we will modify it in place

    # Embed one-hot tokens
    e = self.input_embed(x)  # shape == (batch_size, context_size, embedding_size)
    # Note: ks.layers.Embedding does the same thing but more efficiently for large vocabularies

    if self.use_cnn:
      # 1D convolution across time puts neighbor information into each embedding in the sequence 
      e = self.conv1d(e)

    # Recurrent cell stack
    outputs = []
    # loop over time
    for t, h in enumerate(tf.unstack(e, axis=1)):
      # loop over cells
      for l, cell in enumerate(self.cells):
        h, s[l] = cell(h, s[l])
      outputs.append(h)

    # Feed forward stack
    h = tf.stack(outputs, axis=1)  # stack along the time axis
    for layer in self.output_stack:
      h = layer(h)
    return h, s, None  # logits, final hidden state, cache

# Training loop

In [ ]:
learning_rate = 1e-3

model = RNN(use_cnn=True)
# model = AttentionRNN(use_attn=True, use_cnn=True)
# model = Transformer(d_model=128, num_layers=4, num_attn_heads=8)


loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,  # predictions will be given as logits (log unnormalized probabilities) rather than probabilities
)

# optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.0)
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Use GPU if available.
# https://www.tensorflow.org/guide/gpu
GPUs = tf.config.list_physical_devices('GPU')
device = '/GPU:0' if GPUs else '/CPU:0'
print('device =', device)

In [ ]:
@tf.function
def train_step(batch, labels, state=None):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    logits, state_out, _ = model(batch, state, training=True)
    loss = loss_object(labels, logits)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss, logits, state_out


@tf.function
def accuracy(logits, target, normalize=True):
  argmaxs = tf.math.argmax(logits, axis=-1)
  corrects = tf.math.equal(argmaxs, target)
  if normalize:
    return tf.reduce_mean(tf.cast(corrects, tf.float32))
  else:
    return tf.math.count_nonzero(corrects)

In [ ]:
%rm -rf logs

In [ ]:
# Start tensorboard (optional)
# This will embed a tensorboard front-end in the output of this cell, which will display training graphs in realtime.
# See https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
tb_writer = tf.summary.create_file_writer('logs')  # Tensorboard writer
global_step = 0

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
  # Performing truncated backprop through time (TBPTT).
  # States are carried over between batches, but gradients are not propagated beyond a batch.
  # At the end of each epoch the state is reset to its default (typically all zeros).
  state = None  # None tells the model to use the default state
  for batch_i in range(NUM_BATCHES):  
    # Move tensors to the configured device
    batch = get_batch(batch_i)
    labels = get_batch(batch_i, offset=1)
    with tf.device(device):
      loss_, logits_, state = train_step(batch, labels, state)
    
    global_step += 1

    if batch_i % 10 == 0:
      loss_ = loss_.numpy()
      acc_ = accuracy(logits_, labels).numpy()
      print('  Step: %d out of %d | Train Loss: %.4f | Train Accuracy: %.2f' % (batch_i, NUM_BATCHES, loss_, acc_))
      with tb_writer.as_default():
        tf.summary.scalar('train_loss', loss_, step=global_step)
        tf.summary.scalar('train_accuracy', acc_, step=global_step)

  # Save model checkpoint
  # model.save(f'./training_checkpoints/ckpt_{epoch}')

  print('')
  print('Finished epoch')
  print('')

In [ ]:
# Manual save model
model.save(f'./training_checkpoints/ckpt_{epoch}')

In [ ]:
# Inspect predictions
batch_num = 1000
input_num = 0
l, _, _ = model(get_batch(batch_num, offset=0))
p = tf.nn.softmax(l[input_num], axis=-1).numpy()
print('predictions:')
print(np.argmax(p, axis=1))
print(token_to_ascii[np.argmax(p, axis=1)].tobytes().decode('utf8'))
print('\ntargets:')
print(get_batch(batch_num, offset=1)[input_num])
print(token_to_ascii[get_batch(batch_num, offset=1)[input_num]].tobytes().decode('utf8'))

## Load checkpoint

Reference: https://www.tensorflow.org/guide/keras/save_and_serialize

In [ ]:
%ls training_checkpoints

In [ ]:
model_copy = ks.models.load_model('./training_checkpoints/ckpt_0')
model_copy.compile()

# Generate text

In [ ]:
MODE = 'sample'  # 'argmax'
GENERATE_LENGTH = 100
PROMPT = """ROMEO:"""

# Sampling temperature.
# Lower temperature means peakier distribution.
# As the temp goes to 0, the distribution approaches one-hot (equivalent to taking the argmax)
temp = .1

prompt = np.frombuffer(bytes(PROMPT, 'utf-8'), dtype=np.uint8)
prompt = ascii_to_token[prompt]
x = prompt[None, :]  # Last token in prompt is the input to the first generating step
_, state, cache = model(x[:, :-1], training=False)  # get state and cache for prompt
for n in range(GENERATE_LENGTH):
  logits, state, cache = model(x[:, -1:], state, cache, training=False)
  if MODE == 'sample':
    next = tf.random.categorical(logits[:, -1]/temp, num_samples=1)
  else:  # MODE == 'argmax'
    next = tf.math.argmax(logits[:, -1:], axis=-1)
  x = np.concatenate((x, next.numpy()), axis=1)

In [ ]:
print(token_to_ascii[x].tobytes().decode('utf8'))

In [ ]:
# TODO: implement beam search

# Fancy Language Models

Additional bells, whistles, and "exotic" auto-regressive architectures:
- Attention + recurrent cells
- Transformer (attention only)
- WaveNet (convolution only)

Scaled dot-product attention from [Attention Is All You Need](https://arxiv.org/abs/1706.03762):

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

where $Q$ is a matrix with shape $s_q \times d_q$,  
$K$ with shape $s_k \times d_q$,  
and $V$ with shape $s_k \times d_v$,  
with $s_q, s_k$ the sequence lengths of queries and keys,  
and $d_q, d_v$ are the vector lengths (depths) of queries (and keys) and values.

    
$QK^T$ has shape $s_q \times s_k$  
and $\text{Attention}(Q, K, V)$ has shape $s_q \times d_v$.

In [ ]:
# Dot product attention is used in "Attention Is All You Need" (https://arxiv.org/abs/1706.03762), which introduces the transformer.

# This implementation is based on tensor2tensor.layers.common_attention.dot_product_attention,
# https://github.com/tensorflow/tensor2tensor/blob/ef1fccebe8d2c0cf482f41f9d940e2938c816c78/tensor2tensor/layers/common_attention.py#L1602
# and the Transformer tutorial,
# https://www.tensorflow.org/text/tutorials/transformer#scaled_dot_product_attention

def dot_product_attention(q, k, v, mask=None, scaled=True, return_weights=False):
  # q.shape == (..., seq_len_q, depth_qk)
  # k.shape == (..., seq_len_kv, depth_qk)
  # v.shape == (..., seq_len_kv, depth_v)
  # mask.shape == (..., seq_len_q, seq_len_kv)
  # mask is a binary tensor, with 0 entries being removed from attention.

  # For each query vector in the query sequence q, we dot product with every key
  # vector in the key sequence k to get attention weights, one for each value
  # vector in the value sequence v. We return the weighted average of the value
  # vectors. Each query vector in the query sequence produces an average value
  # vector, averaged over the same value sequence.

  # If q, k, v have higher ranks than 2, `dot_product_attention` is performed
  # element-wise along the preceding tensor dimensions.

  # Attention logits are unscaled log-weights 
  # logits == [dot(vec_q, vec_k) for (vec_q, vec_k) in zip(q, k)]
  logits = tf.matmul(q, k, transpose_b=True)  # shape == (..., seq_len_q, seq_len_kv)

  if scaled:
    # scale logits by 1/sqrt(depth_qk)
    depth_qk = tf.cast(tf.shape(k)[-1], tf.float32)
    logits = logits / tf.math.sqrt(depth_qk)

  # Mask out sequence elements by zeroing out their attention weights.
  # Attention weights are zero if their corresponding logits are -inf.
  if mask is not None:
    # Use a large negative finite number to avoid getting NaNs.
    logits = tf.where(mask, logits, tf.experimental.numpy.full_like(logits, -1e9))

  # Turn attention logits into non-negative weights that sum to 1
  weights = tf.nn.softmax(logits, axis=-1)  # shape == (..., seq_len_q, seq_len_kv)

  # Take weighted average
  average_v = tf.matmul(weights, v)  # shape == (..., seq_len_q, depth_v)

  if return_weights:
    return average_v, weights
  return average_v

In [ ]:
CELL_CLS = {
    'rnn': ks.layers.SimpleRNNCell,
    'lstm': ks.layers.LSTMCell,
    'gru': ks.layers.GRUCell,
}['lstm']

## Attention + recurrent cells

In [ ]:
# RNN cell on top of an attention layer.

class AttentionRNN(ks.Model):

  def __init__(self, use_attn=False, use_cnn=False):
    super(AttentionRNN, self).__init__()
    self.embedding_size = 20
    self.input_embed = ks.layers.Dense(self.embedding_size)
    self.cells = [CELL_CLS(100)]  # , CELL_CLS(50)]
    self.output_stack = [ks.layers.Dense(VOCAB_SIZE)]
    self.conv1d = ks.layers.Conv1D(filters=self.embedding_size, kernel_size=4, padding='causal')  # 'causal' convolutions only depend on inputs to the left (and center) of the current position
    self.num_attn_heads = 10
    self.query_embed = ks.layers.Dense(self.num_attn_heads * self.embedding_size//2)  # self.embedding_size//2 is the size of the query and key vectors
    self.use_attn = use_attn
    self.use_cnn = use_cnn

  def call(self, x, s=None, cache=None):
    # `x` is the input tensor and `s` is the recurrent state.

    # Expecting x.shape == (batch_size, context_size), where batch_size and context_size can be variable from run to run
    bs, cs = tf.unstack(tf.shape(x))
    x = tf.one_hot(x, VOCAB_SIZE)  # shape == (batch_size, context_size, VOCAB_SIZE), where VOCAB_SIZE is a global constant

    if s is None:
      s = [cell.get_initial_state(batch_size=bs, dtype=tf.float32) for cell in self.cells]
    else:
      s = list(s)  # Make a copy of the input list since we will modify it in place

    # Embed one-hot tokens
    e = self.input_embed(x)  # shape == (batch_size, context_size, embedding_size)
    # Note: ks.layers.Embedding does the same thing but more efficiently for large vocabularies

    if self.use_cnn:
      # 1D convolution across time puts neighbor information into each embedding in the sequence 
      e = self.conv1d(e)

    if self.use_attn:
      # Split embedding dimension into two sectors: key and value.
      # That gives us a key and value pair for each timestep.
      num_features = tf.shape(e)[-1]
      k = e[:, :, :num_features//2]
      v = e[:, :, num_features//2:]
      k_size = tf.shape(k)[-1]  # should equal self.embedding_size//2

      # kv caching
      if cache is not None:
        # prepend cached k and v to new k and v
        k_cache, v_cache = cache
        k = tf.concat((k_cache, k), axis=1)
        v = tf.concat((v_cache, v), axis=1)
        cache_len = tf.shape(k_cache)[1]  # time dimension
      else:
        cache_len = 0
      
      # cache full k and v
      cache = (k, v)

    # Recurrent cell stack
    outputs = []
    # loop over time
    for t, h in enumerate(tf.unstack(e, axis=1)):
      if self.use_attn:
        # Query is computed from the current input and recurrent states.
        query_context = tf.concat(tf.nest.flatten([h, s]), axis=1)
        q = tf.reshape(self.query_embed(query_context), (bs, self.num_attn_heads, k_size))

        # We slice `k` and `v` so that the future is not included
        attn_result = dot_product_attention(q, k[:, :cache_len+t+1], v[:, :cache_len+t+1])
        #    , bias=None, make_image_summary=False)  # Additional arguments for tensor2tensor dot_product_attention

        # attn_result shape is (batch_size, self.num_attn_heads, k_size)
        attn_result = tf.reshape(attn_result, (bs, -1))  # flatten last two dims
        h = tf.concat((h, attn_result), axis=-1)  # Concat current step input embedding with attention result

      # loop over cells
      for l, cell in enumerate(self.cells):
        h, s[l] = cell(h, s[l])
      outputs.append(h)

    # Feed forward stack
    h = tf.stack(outputs, axis=1)  # stack along the time axis
    for layer in self.output_stack:
      h = layer(h)
    return h, s, cache

## Transformer (attention only)

In [ ]:
# Multi-head attention layer.
# This implementation is based on https://www.tensorflow.org/text/tutorials/transformer#multi-head_attention

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, *, d_out, d_attn, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_attn = d_attn
    self.d_out = d_out

    self.d_concat = d_attn * num_heads

    self.w_q = tf.keras.layers.Dense(self.d_concat)
    self.w_k = tf.keras.layers.Dense(self.d_concat)
    self.w_v = tf.keras.layers.Dense(self.d_concat)
    self.w_o = tf.keras.layers.Dense(self.d_out)

  def split_heads(self, x):
    # x.shape == (batch_size, seq_len, d_concat)
    # Split the last dimension into (num_heads, d_attn).
    # Transpose the result such that the shape is (batch_size, num_heads, seq_len, d_attn)
    batch_size, seq_len, _ = tf.unstack(tf.shape(x))
    x = tf.reshape(x, (batch_size, seq_len, self.num_heads, self.d_attn))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, q, k, v, mask):
    # q.shape == (batch_size, seq_len, d_q)
    # k.shape == (batch_size, seq_len, d_k)
    # v.shape == (batch_size, seq_len, d_v)
    batch_size = tf.shape(q)[0]

    q = self.w_q(q)  # (batch_size, seq_len, d_concat)
    k = self.w_k(k)  # (batch_size, seq_len, d_concat)
    v = self.w_v(v)  # (batch_size, seq_len, d_concat)

    q = self.split_heads(q)  # (batch_size, num_heads, seq_len_q, d_attn)
    k = self.split_heads(k)  # (batch_size, num_heads, seq_len_k, d_attn)
    v = self.split_heads(v)  # (batch_size, num_heads, seq_len_v, d_attn)
    # For our purposes seq_len_q == seq_len_k == seq_len_v, but it's insightful
    # to see how these variables propagate if they were not equal.

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, d_attn)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    attn_avg, attn_weights = dot_product_attention(q, k, v, mask, return_weights=True)
    attn_avg = tf.transpose(attn_avg, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, d_attn)
    attn_avg_concat = tf.reshape(attn_avg, (batch_size, -1, self.d_concat))  # (batch_size, seq_len_q, d_concat)

    output = self.w_o(attn_avg_concat)  # (batch_size, seq_len_q, d_out)

    return output, attn_weights

In [ ]:
# Copied from https://www.tensorflow.org/text/tutorials/transformer#create_the_transformer_model

def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
# The transformer, i.e. self-attention, is introduced in "Attention Is All You Need" (https://arxiv.org/abs/1706.03762).

# We are only interested in the self-attntion decoder, like the one used in 
# GPT-1 (https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
# and "Generating wikipedia by summarizing long sequences" (https://arxiv.org/abs/1801.10198).

# This implementation is based on
# https://www.tensorflow.org/text/tutorials/transformer#decoder_layer
# https://www.tensorflow.org/text/tutorials/transformer#decoder
# and https://www.tensorflow.org/text/tutorials/transformer#create_the_transformer_model


SelfAttnLayer = namedtuple('SelfAttnLayer', 'mha,layernorm1,dropout1,ff,layernorm2,dropout2')


class Transformer(ks.Model):
  # Maximum supported sequence length.
  # Make this large enough to handle evaluation inputs.
  MAX_SEQ_LENGTH = 10*CONTEXT_SIZE


  def __init__(self, d_model, num_layers, num_attn_heads, dropout_rate=0.1):
    super(Transformer, self).__init__()

    self.pos_encoding = positional_encoding(self.MAX_SEQ_LENGTH, d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.dense_bottom = tf.keras.layers.Dense(d_model)

    self.self_attention_layers = [
        SelfAttnLayer(
            mha = MultiHeadAttention(d_out=d_model, d_attn=d_model // num_attn_heads, num_heads=num_attn_heads),
            layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6),
            dropout1 = tf.keras.layers.Dropout(dropout_rate),

            ff = tf.keras.Sequential([
                tf.keras.layers.Dense(256, activation='relu'),  # (batch_size, seq_len, d_ff)
                tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
            ]),
            layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6),
            dropout2 = tf.keras.layers.Dropout(dropout_rate),
        )
        for _ in range(num_layers)
    ]

    self.dense_top = tf.keras.layers.Dense(VOCAB_SIZE)

  def call(self, x, state=None, cache=None, training=False, return_attn_weights=False):
    # x.shape == (batch_size, seq_len)

    # Expecting x.shape == (batch_size, seq_len), where batch_size and seq_len can be variable from run to run
    bs, seq_len = tf.unstack(tf.shape(x))
    x = tf.one_hot(x, VOCAB_SIZE)  # shape == (batch_size, seq_len, VOCAB_SIZE), where VOCAB_SIZE is a global constant
    x = self.dense_bottom(x)  # shape == (batch_size, seq_len, d_model)

    # get the number of time steps stored in the cache
    if cache is not None:
      cache_seq_len = tf.shape(cache[0])[1]
    else:
      cache_seq_len = 0

    look_ahead_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    look_ahead_mask = tf.concat((tf.ones((seq_len, cache_seq_len)), look_ahead_mask), axis=1)
    look_ahead_mask = tf.cast(look_ahead_mask, tf.bool)
    look_ahead_mask = tf.expand_dims(tf.expand_dims(look_ahead_mask, axis=0), axis=0)  # shape == (1, 1, seq_len, seq_len + cache_seq_len)
    
    # self-attention decoder
    attention_weights = {}
    x += self.pos_encoding[:, cache_seq_len:cache_seq_len+seq_len, :]
    x = self.dropout(x, training=training)

    output_cache = []
    for i, (mha, layernorm1, dropout1, ff, layernorm2, dropout2) in enumerate(self.self_attention_layers):
      # kv caching
      q = x  # keep queries separate
      if cache is not None:
        kv = tf.concat((cache[i], x), axis=1)  # concat along time axis
      else:
        kv = x
      output_cache.append(kv)

      attn_result, attn_weights_block = mha(q, kv, kv, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
      attn_result = dropout1(attn_result, training=training)
      # residual connction and layer norm
      x = layernorm1(attn_result + x)  # (batch_size, target_seq_len, d_model)

      ff_output = ff(x)  # (batch_size, target_seq_len, d_model)
      ff_output = dropout2(ff_output, training=training)
      x = layernorm2(ff_output + x)  # (batch_size, target_seq_len, d_model)

      attention_weights[f'decoder_layer{i+1}_block'] = attn_weights_block
    
    # dense layer on output
    final_output = self.dense_top(x)  # (batch_size, tar_seq_len, target_vocab_size)

    return_val = (final_output, None, output_cache)
    if return_attn_weights:
      return_val += (attention_weights,)
    return return_val

## WaveNet (convolution only)

In [ ]:
# TODO